In [ ]:
# I'm sorry sir I can't uplaod it on live server(streamlit) due to work load. I'm building a project that's why I'm so busy but I Know how to deploy on streamlit.

In [2]:
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

In [3]:
dataset = load_dataset("ag_news")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [4]:
train_data = dataset["train"]
test_data = dataset["test"]

print(f"Train samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")
print(f"Labels: {train_data.features['label'].names}")

Train samples: 120000
Test samples: 7600
Labels: ['World', 'Sports', 'Business', 'Sci/Tech']


In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_test = test_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [6]:
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_test = tokenized_test.rename_column("label", "labels")

tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)   
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

In [9]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,    
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
eval_results = trainer.evaluate()
print(f"Accuracy: {eval_results['eval_accuracy']:.4f}")
print(f"F1-Score: {eval_results['eval_f1']:.4f}")

In [ ]:
model.save_pretrained("./news_classifier_bert")
tokenizer.save_pretrained("./news_classifier_bert")

In [ ]:
def predict_news_category(headline):
    inputs = tokenizer(headline, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=-1).item()
    
    labels = ['World', 'Sports', 'Business', 'Sci/Tech']
    confidence = predictions[0][predicted_class].item()
    
    return {
        "category": labels[predicted_class],
        "confidence": confidence,
        "all_scores": {labels[i]: predictions[0][i].item() for i in range(4)}
    }

In [ ]:
test_headlines = [
    "Stock market reaches new highs as tech companies surge",
    "Football championship finals scheduled for next month",
    "Scientists discover new species in Amazon rainforest",
    "International summit addresses climate change concerns"
]

for headline in test_headlines:
    result = predict_news_category(headline)
    print(f"Headline: {headline}")
    print(f"Predicted: {result['category']} (Confidence: {result['confidence']:.3f})")
    print("-")